In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

from MazeGenerationAlgorithms.HuntAndKill import HuntAndKillMaze
from MazeGenerationAlgorithms.DepthFirstPrim import DepthFirstPrimMaze
from MazeGenerationAlgorithms.RandomizedKruskal import RandomizedKruskalMaze
from MazeGenerationAlgorithms.Sidewinder import SidewinderMaze
from MazeGenerationAlgorithms.StochasticPrim import StochasticPrimMaze
from MazeGenerationAlgorithms.Wilson import WilsonMaze
from MazeGenerationAlgorithms.RandomizedPrim import RandomizedPrimMaze
from MazeGenerationAlgorithms.DFS import DFSMaze
from MazeGenerationAlgorithms.RecursiveBacktracker import RecursiveBacktrackerMaze
from MazeGenerationAlgorithms.AldousBroder import AldousBroderMaze

import pandas as pd
import networkx as nx

In [2]:
class DataExporter:
    def __init__(self, maze_objects):
        self.mazes = maze_objects
        self.stats_df = self._analyze_mazes()

    def _analyze_mazes(self):
        stats = []
        
        for maze in self.mazes:
            degrees = dict(maze.maze.degree())
            solution_path = nx.shortest_path(maze.maze, maze.start, maze.end)
            solution_set = set(solution_path)
            
            sol_len = len(solution_path) - 1
            manhattan = abs(maze.end[0] - maze.start[0]) + abs(maze.end[1] - maze.start[1])
            tortuosity = sol_len / manhattan if manhattan > 0 else 1
            
            deadends = [node for node, degree in degrees.items() 
                       if degree == 1 and node not in [maze.start, maze.end]]
            
            intersections = [node for node, degree in degrees.items() if degree > 2]
            intersection_degree_sum = sum(degrees[node] for node in intersections)
            
            avg_intersection_degree = round(intersection_degree_sum / len(intersections), 2) if intersections else 0
            max_intersection_degree = max(degrees[node] for node in intersections) if intersections else 0
            
            dist_from_solution = self._get_node_to_deadend_distances(maze.maze, solution_set, deadends)
            
            intersection_set = set(intersections)
            deadend_crossroads = sum(1 for d in deadends 
                                   if next(iter(maze.maze.neighbors(d))) in intersection_set)
            
            deadend_lengths = []
            for deadend in deadends:
                neighbor = next(iter(maze.maze.neighbors(deadend)))
                if neighbor in intersection_set:
                    deadend_lengths.append(1) 
                else:
                    try:
                        path = nx.shortest_path(maze.maze, deadend, 
                                             [n for n in intersection_set if n != neighbor][0])
                        deadend_lengths.append(len(path) - 1)
                    except (nx.NetworkXNoPath, IndexError):
                        continue
            
            avg_deadend_length = round(sum(deadend_lengths) / len(deadend_lengths), 2) if deadend_lengths else 0
            
            # Calculate tortuosity for paths from solution to deadends
            deadend_tortuosities = []
            for src in solution_set:
                for tgt in deadends:
                    try:
                        path = nx.shortest_path(maze.maze, src, tgt)
                        path_len = len(path) - 1
                        path_manhattan = abs(tgt[0] - src[0]) + abs(tgt[1] - src[1])
                        if path_manhattan > 0:
                            deadend_tortuosities.append(path_len / path_manhattan)
                    except nx.NetworkXNoPath:
                        continue
            
            avg_deadend_tortuosity = round(sum(deadend_tortuosities) / len(deadend_tortuosities), 2) if deadend_tortuosities else 0
            
            stats.append({
                'Maze Type': maze.maze_name.replace(' Algorithm', '').strip(),
                'Size': f"{maze.width}x{maze.height}",
                '# of Intersections': len(intersections),
                'Avg Intersection Degree': avg_intersection_degree,
                'Max Intersection Degree': max_intersection_degree,
                'Solution Length': sol_len,
                'Solution Path Tortuosity': round(tortuosity, 2),
                'Avg Distance to Deadend(From Solution Path) Tortuosity': avg_deadend_tortuosity,
                '# of Deadends': len(deadends),
                'Deadend Crossroads': deadend_crossroads,
                'Avg Distance to Deadend(From Solution Path)': self._average_distance(dist_from_solution),
                'Avg Deadend Length(from Intersection)': avg_deadend_length,
            })

        return pd.DataFrame(stats)

    def _get_node_to_deadend_distances(self, graph, nodes, deadends):
        if not nodes or not deadends:
            return {}
            
        distances = {}
        for src in nodes:
            try:
                src_paths = nx.single_source_shortest_path_length(graph, src)
                for tgt in deadends:
                    if tgt in src_paths:
                        distances[f"{src}-{tgt}"] = src_paths[tgt]
            except nx.NetworkXNoPath:
                continue
                    
        return distances

    def _average_distance(self, dist_map):
        return round(sum(dist_map.values()) / len(dist_map), 2) if dist_map else 0

    def _maximum_distance(self, dist_map):
        return max(dist_map.values(), default=0)

In [ ]:
width, height = 25, 25
sample_size = 100

maze_groups = [
    [SidewinderMaze, RandomizedPrimMaze],
    [DepthFirstPrimMaze, StochasticPrimMaze],
    [RandomizedKruskalMaze],
    [WilsonMaze, AldousBroderMaze],
    [HuntAndKillMaze, DFSMaze, RecursiveBacktrackerMaze]
]

all_data = []
for maze_group in maze_groups:
    maze_lst = []
    for _ in range(sample_size):
        maze_lst.extend([maze_class(width, height) for maze_class in maze_group])
    
    data = DataExporter(maze_lst)
    all_data.append(data.stats_df)

data_ = pd.concat(all_data, ignore_index=True)
data_.to_csv('metric.csv', index=False)